In [10]:

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../fastai/") # go to parent dir

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

In [13]:
PATH='data/desPres/'

TRN_PATH = 'train/'
VAL_PATH = 'test/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

# %ls {PATH}

In [16]:
import os

NameError: name 'dst' is not defined

In [21]:
src = './jos'

In [29]:
for fname in os.listdir(src):
      if os.path.isfile(os.path.join(src, fname)):
            if random.random()>0.8:
                os.rename(os.path.join(src, fname), os.path.join(src+'/train', fname))
            else:
                os.rename(os.path.join(src, fname), os.path.join(src+'/test', fname))

In [24]:
import random

In [28]:
random.random()

0.5716794185408048

In [30]:
len(os.listdir("./jos/train"))

76

In [31]:
len(os.listdir("./jos/test"))

358